##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 你好，量子世界

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://tensorflow.google.cn/quantum/tutorials/hello_many_worlds"><img src="https://tensorflow.google.cn/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/zh-cn/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://tensorflow.google.cn/images/colab_logo_32px.png">在 Google Colab 中运行</a></td>
  <td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/zh-cn/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://tensorflow.google.cn/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a></td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/zh-cn/quantum/tutorials/hello_many_worlds.ipynb"><img src="https://tensorflow.google.cn/images/download_logo_32px.png">下载笔记本</a></td>
</table>

本教程介绍经典神经网络如何学习纠正量子位校准误差。其中介绍了 <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>（一个用于创建、编辑和调用嘈杂中型量子 (NISQ) 电路的 Python 框架），并演示了 Cirq 如何通过接口与 TensorFlow Quantum 连接。

## 设置

In [ ]:
!pip install tensorflow==2.4.1

安装 TensorFlow Quantum：

In [ ]:
!pip install tensorflow-quantum

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

现在，导入 TensorFlow 和模块依赖项：

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 1. 基础知识

### 1.1 Cirq 和参数化量子电路

在研究 TensorFlow Quantum (TFQ) 之前，我们先来了解 <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a> 的一些基础知识。Cirq 是 Google 开发的一个用于量子计算的 Python 库。您可以使用它定义电路，包括静态和参数化门。

Cirq 使用 <a target="_blank" href="https://www.sympy.org" class="external">SymPy</a> 符号表示自由参数。

In [ ]:
a, b = sympy.symbols('a b')

下面的代码可以使用您的参数创建一个双量子位电路：

In [ ]:
# Create two qubits
q0, q1 = cirq.GridQubit.rect(1, 2)

# Create a circuit on these qubits using the parameters you created above.
circuit = cirq.Circuit(
    cirq.rx(a).on(q0),
    cirq.ry(b).on(q1), cirq.CNOT(control=q0, target=q1))

SVGCircuit(circuit)

要评估电路，您可以使用 `cirq.Simulator` 接口。通过传入 `cirq.ParamResolver` 对象，您可以将电路中的自由参数替换为具体的数字。下面的代码可以计算参数化电路的原始状态向量输出：

In [ ]:
# Calculate a state vector with a=0.5 and b=-0.5.
resolver = cirq.ParamResolver({a: 0.5, b: -0.5})
output_state_vector = cirq.Simulator().simulate(circuit, resolver).final_state_vector
output_state_vector

如果脱离模拟环境，您将无法直接访问状态向量（请注意上面的输出中的复杂数字）。为了达到物理真实感，您必须指定一个测量值，将状态向量转换为经典计算机可以理解的实数。Cirq 使用 <a target="_blank" href="https://en.wikipedia.org/wiki/Pauli_matrices" class="external">Pauli 算子</a> $\hat{X}$、$\hat{Y}$ 和 $\hat{Z}$ 的组合指定测量值。例如，以下代码可在您刚刚模拟的状态向量上测量 $\hat{Z}_0$ 和 $\frac{1}{2}\hat{Z}_0 + \hat{X}_1$。

In [ ]:
z0 = cirq.Z(q0)

qubit_map={q0: 0, q1: 1}

z0.expectation_from_state_vector(output_state_vector, qubit_map).real

In [ ]:
z0x1 = 0.5 * z0 + cirq.X(q1)

z0x1.expectation_from_state_vector(output_state_vector, qubit_map).real

### 1.2 作为向量的量子电路

TensorFlow Quantum (TFQ) 提供了 `tfq.convert_to_tensor`，后者是一个可以将 Cirq 对象转换为张量的函数。这样，您可以将 Cirq 对象发送到我们的<a target="_blank" href="https://tensorflow.google.cn/quantum/api_docs/python/tfq/layers">量子层</a>和<a target="_blank" href="https://tensorflow.google.cn/quantum/api_docs/python/tfq/get_expectation_op">量子运算</a>。可以在 Cirq Circuit 和 Cirq Paulis 的列表或数组中调用该函数：

In [ ]:
# Rank 1 tensor containing 1 circuit.
circuit_tensor = tfq.convert_to_tensor([circuit])

print(circuit_tensor.shape)
print(circuit_tensor.dtype)

这可以将 Cirq 对象编码为 `tf.string` 张量，`tfq` 运算在需要时可以对这些张量进行解码。

In [ ]:
# Rank 1 tensor containing 2 Pauli operators.
pauli_tensor = tfq.convert_to_tensor([z0, z0x1])
pauli_tensor.shape

### 1.3 电路模拟批处理

TFQ 提供了计算期望值、样本和状态向量的方法。目前，我们先重点介绍*期望值*。

计算期望值的最高级接口是 `tfq.layers.Expectation` 层，它是一个 `tf.keras.Layer`。简而言之，该层等效于通过许多 `cirq.ParamResolvers` 模拟参数化电路；不过，TFQ 允许对随后的 TensorFlow 语义进行批处理，而电路则使用高效的 C++ 代码进行模拟。

创建一组替代 `a` 和 `b` 参数的值：

In [ ]:
batch_vals = np.array(np.random.uniform(0, 2 * np.pi, (5, 2)), dtype=np.float32)

在 Cirq 中通过参数值对电路执行进行批处理需要一个循环：

In [ ]:
cirq_results = []
cirq_simulator = cirq.Simulator()

for vals in batch_vals:
    resolver = cirq.ParamResolver({a: vals[0], b: vals[1]})
    final_state_vector = cirq_simulator.simulate(circuit, resolver).final_state_vector
    cirq_results.append(
        [z0.expectation_from_state_vector(final_state_vector, {
            q0: 0,
            q1: 1
        }).real])

print('cirq batch results: \n {}'.format(np.array(cirq_results)))

TFQ 中简化了同一运算：

In [ ]:
tfq.layers.Expectation()(circuit,
                         symbol_names=[a, b],
                         symbol_values=batch_vals,
                         operators=z0)

## 2. 混合量子-经典优化

现在，您已经了解基础知识。接下来，我们使用 TensorFlow Quantum 构造一个*混合量子-经典神经网络*。您将训练一个经典神经网络来控制单个量子位。我们将优化控制，以便将量子位正确准备为 `0` 或 `1` 状态，克服模拟系统校准误差。下面是架构图：

<img src="./images/nn_control1.png" width="1000">

即使没有神经网络，这也是一个很容易解决的问题，但主题与您使用 TFQ 可能解决的实际量子控制问题类似。它使用 `tf.keras.Model` 中的 `tfq.layers.ControlledPQC`（参数化量子电路）层演示了一个端到端量子-经典计算示例。

对于本教程中的实现，此架构分为以下三部分：

- *输入电路*或*数据点电路*：前三个 $R$ 门。
- *受控电路*：另外三个 $R$ 门。
- *控制器*：设置受控电路参数的经典神经网络。

### 2.1 受控电路定义

如上图所示，定义可学习的单个位旋转。这与我们的受控电路对应。

In [ ]:
# Parameters that the classical NN will feed values into.
control_params = sympy.symbols('theta_1 theta_2 theta_3')

# Create the parameterized circuit.
qubit = cirq.GridQubit(0, 0)
model_circuit = cirq.Circuit(
    cirq.rz(control_params[0])(qubit),
    cirq.ry(control_params[1])(qubit),
    cirq.rx(control_params[2])(qubit))

SVGCircuit(model_circuit)

### 2.2 控制器

现在，定义控制器网络： 

In [ ]:
# The classical neural network layers.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

给定一批命令，控制器就会输出受控电路的一批控制信号。

控制器是随机初始化的，所以这些输出目前没有用处。

In [ ]:
controller(tf.constant([[0.0],[1.0]])).numpy()

### 2.3 将控制器连接到电路

使用 `tfq` 将控制器作为单个 `keras.Model` 连接到受控电路。

请参阅 [Keras 函数式 API 指南](https://tensorflow.google.cn/guide/keras/functional)详细了解这种样式的模型定义。

首先，定义模型的输入： 

In [ ]:
# This input is the simulated miscalibration that the model will learn to correct.
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.string,
                                name='circuits_input')

# Commands will be either `0` or `1`, specifying the state to set the qubit to.
commands_input = tf.keras.Input(shape=(1,),
                                dtype=tf.dtypes.float32,
                                name='commands_input')


接着，将运算应用到这些输入来定义计算。

In [ ]:
dense_2 = controller(commands_input)

# TFQ layer for classically controlled circuits.
expectation_layer = tfq.layers.ControlledPQC(model_circuit,
                                             # Observe Z
                                             operators = cirq.Z(qubit))
expectation = expectation_layer([circuits_input, dense_2])

现在，将此计算打包成 `tf.keras.Model`：

In [ ]:
# The full Keras model is built from our layers.
model = tf.keras.Model(inputs=[circuits_input, commands_input],
                       outputs=expectation)

该网络架构如下面的模型图所示。将模型图与架构图进行对比可验证其正确性。

注：可能需要安装了 `graphviz` 软件包的系统。

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=70)

此模型需要两个输入：控制器的命令，以及控制器尝试纠正其输出的输入电路。 

### 2.4 数据集

该模型会尝试为每个命令的 $\hat{Z}$ 输出正确的测量值。命令和正确值的定义如下。

In [ ]:
# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired Z expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

这并不是此任务的完整训练数据集。数据集中的每个数据点还需要一个输入电路。

### 2.4 输入电路定义

下面的输入电路定义该模型将学习纠正的随机校准误差。

In [ ]:
random_rotations = np.random.uniform(0, 2 * np.pi, 3)
noisy_preparation = cirq.Circuit(
  cirq.rx(random_rotations[0])(qubit),
  cirq.ry(random_rotations[1])(qubit),
  cirq.rz(random_rotations[2])(qubit)
)
datapoint_circuits = tfq.convert_to_tensor([
  noisy_preparation
] * 2)  # Make two copied of this circuit

电路有两个副本，每个数据点一个。

In [ ]:
datapoint_circuits.shape

### 2.5 训练

利用定义的输入，您可以试运行 `tfq` 模型。

In [ ]:
model([datapoint_circuits, commands]).numpy()

现在，请运行标准训练流程，针对 `expected_outputs` 调整这些值。

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)
history = model.fit(x=[datapoint_circuits, commands],
                    y=expected_outputs,
                    epochs=30,
                    verbose=0)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

从此图中您可以看到，神经网络已经学会解决系统校准错误。

### 2.6 验证输出

现在，使用训练的模型来纠正量子位校准误差。对于 Cirq：

In [ ]:
def check_error(command_values, desired_values):
  """Based on the value in `command_value` see how well you could prepare
  the full circuit to have `desired_value` when taking expectation w.r.t. Z."""
  params_to_prepare_output = controller(command_values).numpy()
  full_circuit = noisy_preparation + model_circuit

  # Test how well you can prepare a state to get expectation the expectation
  # value in `desired_values`
  for index in [0, 1]:
    state = cirq_simulator.simulate(
        full_circuit,
        {s:v for (s,v) in zip(control_params, params_to_prepare_output[index])}
    ).final_state_vector
    expt = cirq.Z(qubit).expectation_from_state_vector(state, {qubit: 0}).real
    print(f'For a desired output (expectation) of {desired_values[index]} with'
          f' noisy preparation, the controller\nnetwork found the following '
          f'values for theta: {params_to_prepare_output[index]}\nWhich gives an'
          f' actual expectation of: {expt}\n')


check_error(commands, expected_outputs)

在训练期间，损失函数的值可提供模型学习效果的大致情况。损失值越小，以上代码单元中的期望值就越接近 `desired_values`。如果您不太关心参数值，则随时可以使用 `tfq` 检查上面的输出：

In [ ]:
model([datapoint_circuits, commands])

## 3 学习准备不同算子的本征态

您可以随意将 $\pm \hat{Z}$ 本征态与 1 和 0 对应。但为了简便起见，您可以让 1 与 $+ \hat{Z}$ 本征态对应，而让 0 与 $-\hat{X}$ 本征态对应。一种实现方式是为每个命令指定一个不同的测量算子，如下图所示：

<img src="./images/nn_control2.png" width="1000">

这要求使用 <code>tfq.layers.Expectation</code>。现在，您的输入已经包括三个对象：电路、命令和算子。输出仍为期望值。

### 3.1 新模型定义

我们来看看完成此任务的模型：

In [ ]:
# Define inputs.
commands_input = tf.keras.layers.Input(shape=(1),
                                       dtype=tf.dtypes.float32,
                                       name='commands_input')
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.dtypes.string,
                                name='circuits_input')
operators_input = tf.keras.Input(shape=(1,),
                                 dtype=tf.dtypes.string,
                                 name='operators_input')

下面是控制器网络：

In [ ]:
# Define classical NN.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

使用 `tfq` 将电路与控制器合并到单个 `keras.Model` 中：

In [ ]:
dense_2 = controller(commands_input)

# Since you aren't using a PQC or ControlledPQC you must append
# your model circuit onto the datapoint circuit tensor manually.
full_circuit = tfq.layers.AddCircuit()(circuits_input, append=model_circuit)
expectation_output = tfq.layers.Expectation()(full_circuit,
                                              symbol_names=control_params,
                                              symbol_values=dense_2,
                                              operators=operators_input)

# Contruct your Keras model.
two_axis_control_model = tf.keras.Model(
    inputs=[circuits_input, commands_input, operators_input],
    outputs=[expectation_output])

### 3.2 数据集

现在，对于为 `model_circuit` 提供的每个数据点，还要包括要测量的算子：

In [ ]:
# The operators to measure, for each command.
operator_data = tfq.convert_to_tensor([[cirq.X(qubit)], [cirq.Z(qubit)]])

# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

### 3.3 训练

现在，您已经有了新的输入和输出，可以使用 Keras 重新进行训练。

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()

two_axis_control_model.compile(optimizer=optimizer, loss=loss)

history = two_axis_control_model.fit(
    x=[datapoint_circuits, commands, operator_data],
    y=expected_outputs,
    epochs=30,
    verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

损失函数的值已降为零。

`controller` 可作为独立模型提供。您可以调用控制器，并检查其对每个命令信号的响应。要正确对比这些输出与 `random_rotations` 的内容，您可能需要花些功夫。

In [ ]:
controller.predict(np.array([0,1]))

成功：看看您是否可以调整第一个模型中的 `check_error` 函数来使用新模型架构。